# Phase Change Indicator

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("..\\Data\\KOSPI_Daily_1980data.csv", encoding="utf-8")
df.index = pd.to_datetime(df["Date"])
df_close = df["Close"]
df_close = df_close/df_close[0]  #re-scale
KOSPI = pd.DataFrame(df_close, index=df.index)

In [3]:
Period = 20  # 기간
P = Period - 1  # 기간 index
# 모멘텀
KOSPI["Momentum"] = KOSPI - KOSPI.shift(P)

In [4]:
# PCI 구하기
PCI = [0] * P  # PCI 저장할 리스트
for ind in range(P,len(KOSPI)):
    sumUpDi =  0.0
    sumDownDi = 0.0
    Gradient = []
    Deviation = []
    for ind2 in range(Period):
        # ex) 5일 첫 Close + 5일 모멘텀 * 0/4, 1/4, 2/4, 3/4, 4/4
        val2 = KOSPI["Close"].iloc[ind-P] + (KOSPI["Momentum"].iloc[ind]*(ind2)/P)
        Gradient.append(np.round(val2,4))
    for ind3 in range(Period):
        val3 = KOSPI["Close"].iloc[ind-P+ind3] - Gradient[ind3]
        if KOSPI["Close"].iloc[ind-P+ind3] > Gradient[ind3]:
            sumUpDi += abs(val3)
        elif KOSPI["Close"].iloc[ind-P+ind3] < Gradient[ind3]:
            sumDownDi += abs(val3)
        else:
            None
    #ZeroDivisionError 방지코자 if문 사용함
    if sumUpDi != 0:
        a = sumUpDi / (sumUpDi+sumDownDi) * 100
        PCI.append(np.round(a,2))
    else:
        PCI.append(0)
    #print ("sumDownDi {} sumUpDi {}".format(sumDownDi, sumUpDi))
# df에 값 기록하기
KOSPI["PCI"] = PCI

In [5]:
# Momentum > 0 이고, PCI < 20 일때 매수 
# Momentum < 0 이고, PCI > 80 일때 매도 
entry = []
for i in range(len(KOSPI)):
    if KOSPI["Momentum"][i] > 0 and KOSPI["PCI"][i] < 20:
        entry.append("Buy")
    elif KOSPI["Momentum"][i] < 0 and KOSPI["PCI"][i] > 80:
        entry.append("Sell")
    else:
        entry.append("Hold")
KOSPI["Entry"] = entry

In [6]:
KOSPI.tail(30)

,Close,Momentum,PCI,Entry
Date,,,,
2018-12-05,21.0131,0.0868,27.26,Hold
2018-12-06,20.6869,-0.1740,83.02,Sell
2018-12-07,20.7576,-0.0468,82.37,Sell
2018-12-10,20.5379,-0.1744,98.34,Sell
2018-12-11,20.5297,-0.1508,99.07,Sell
2018-12-12,20.8257,-0.0549,56.60,Hold
2018-12-13,20.9555,0.0315,32.80,Hold
2018-12-14,20.6938,-0.3118,54.89,Hold
2018-12-17,20.7109,-0.1149,74.21,Hold
